In [22]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\HeavyNeutrino_trilepton\\HeavyNeutrino_trilepton_M-300_V-0p01_tau_NLO_tauhFilter_TuneCP5_13TeV-madgraph-pythia8"
samples = files_from_dir(local_dir),
i_file = 1
tot_events = 0
file = samples[0][0]
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
print("initial number of events: " + str(len(events)))
events["prompt_e"] = events.Electron[events.Electron.genPartFlav == 1]
events["prompt_m"] = events.Muon[events.Muon.genPartFlav == 1]
match_prompt_tau_to_h = events.Tau.genPartFlav == 5
match_prompt_tau_to_e = events.Tau.genPartFlav == 3
match_prompt_tau_to_m = events.Tau.genPartFlav == 4
match_prompt_tau_prompt_e = events.Tau.genPartFlav == 1
match_prompt_tau_prompt_m = events.Tau.genPartFlav == 2
events["prompt_t"] = events.Tau[match_prompt_tau_to_h|match_prompt_tau_to_e|match_prompt_tau_to_m|match_prompt_tau_prompt_e|match_prompt_tau_prompt_m]


events["prompt_l"] = ak.concatenate([events.prompt_e,events.prompt_m, events.prompt_t], axis = -1)
events = events[ak.num(events.prompt_l) >=3]
print("#events requiring at least 3 prompt leptons: " + str(len(events)))
prev_nb_ev = len(events)
events_beg = events

print("========= ELECTRON SELECTION ====================")
print("number of electron before selection: " + str(len(ak.flatten(events.prompt_e))))
events["prompt_e"] = events.prompt_e[(events.prompt_e.pt > 24.) & (events.prompt_e.mvaFall17V2Iso_WP90 > 0.5)]
print("number of electron after selection: " + str(len(ak.flatten(events.prompt_e))))

events["prompt_l"] = ak.concatenate([events.prompt_e,events.prompt_m, events.prompt_t], axis = -1)
events = events[ak.num(events.prompt_l) >=3]
curr_nb_events = len(events)
minus = int(100*(prev_nb_ev - curr_nb_events)/prev_nb_ev)
print("#events requiring at least 3 prompt leptons: " + str(len(events)) + "(-"+ str(minus)+"%)")
prev_nb_ev= curr_nb_events

print("========= MUON SELECTION ====================")
print("number of muon before selection: " + str(len(ak.flatten(events.prompt_m))))
events["prompt_m"] = events.prompt_m[(events.prompt_m.pt > 24.) & (events.prompt_m.mediumPromptId) & (events.prompt_m.pfRelIso03_all < 0.2) & (np.abs(events.prompt_m.dxy) < 0.005)]
muon1, electron1 = ak.unzip(ak.cartesian([events.prompt_m, events.prompt_e], nested=True))
match1 = ak.any(muon1.jetIdx == electron1.jetIdx, axis=-1, mask_identity=False)
events['prompt_m'] = events.prompt_m[(~(match1))]
print("number of muon after selection: " + str(len(ak.flatten(events.prompt_m))))

events["prompt_l"] = ak.concatenate([events.prompt_e,events.prompt_m, events.prompt_t], axis = -1)
events = events[ak.num(events.prompt_l) >=3]
curr_nb_events = len(events)
minus = int(100*(prev_nb_ev - curr_nb_events)/prev_nb_ev)
print("#events requiring at least 3 prompt leptons: " + str(len(events)) + "(-"+ str(minus)+"%)")
prev_nb_ev= curr_nb_events


print("========= TAU SELECTION ====================")
print("number of taus before selection: " + str(len(ak.flatten(events.prompt_t))))
events["prompt_t"] = events.prompt_t[(events.prompt_t.pt > 20.) & (abs(events.prompt_t.eta) < 2.3)& (events.prompt_t.idDeepTau2017v2p1VSmu > 0.5) & (events.prompt_t.idDeepTau2017v2p1VSe > 0.5) & (events.prompt_t.idDeepTau2017v2p1VSjet >=0.5)]
tau2, electron2 = ak.unzip(ak.cartesian([events.prompt_t, events.prompt_e], nested=True))
match2 = ak.any(tau2.jetIdx == electron2.jetIdx, axis=-1, mask_identity=False)
tau3, muon3 = ak.unzip(ak.cartesian([events.prompt_t, events.prompt_m], nested=True))
match3 = ak.any(tau3.jetIdx == muon3.jetIdx, axis=-1, mask_identity=False)  
events['prompt_t'] = events.prompt_t[((~(match2) & ~(match3)) )]  
print("number of taus after selection: " + str(len(ak.flatten(events.prompt_t))))

events["prompt_l"] = ak.concatenate([events.prompt_e,events.prompt_m, events.prompt_t], axis = -1)
events = events[ak.num(events.prompt_l) >=3]
curr_nb_events = len(events)
minus = int(100*(prev_nb_ev - curr_nb_events)/prev_nb_ev)
print("#events requiring at least 3 prompt leptons: " + str(len(events)) + "(-"+ str(minus)+"%)")
prev_nb_ev= curr_nb_events

events = events[ak.num(events.prompt_l) ==3]
curr_nb_events = len(events)
minus = int(100*(prev_nb_ev - curr_nb_events)/prev_nb_ev)
print("#events requiring exactly 3 prompt leptons: " + str(len(events)) + "(-"+ str(minus)+"%)")
prev_nb_ev= curr_nb_events


print("==============no bjet ================")
jets = events.Jet[events.Jet.pt > 25.]
bjets = jets[jets.btagDeepFlavB > 0.2770]
l1 = events.prompt_l[:,0]
l2 = events.prompt_l[:,1]
l3 = events.prompt_l[:,2]
dr1 = bjets.delta_r(l1)
bjets = bjets[dr1 > 0.5]
dr2 = bjets.delta_r(l2)
bjets = bjets[dr2 > 0.5]
dr3 = bjets.delta_r(l3)
bjets = bjets[dr3 > 0.5]
cut = (ak.num(bjets) == 0)
events = events[cut]
curr_nb_events = len(events)
minus = int(100*(prev_nb_ev - curr_nb_events)/prev_nb_ev)
print("#events requiring 0 non-overlapping bjet: " + str(len(events)) + "(-"+ str(minus)+"%)")
prev_nb_ev= curr_nb_events

print("percentage of left viable events: " + str(int(100*len(events)/len(events_beg))) + "%")



initial number of events: 96498
#events requiring at least 3 prompt leptons: 67100
========= ELECTRON SELECTION ====================
number of electron before selection: 31847
number of electron after selection: 26302
#events requiring at least 3 prompt leptons: 64159(-4%)
========= MUON SELECTION ====================
number of muon before selection: 33665
number of muon after selection: 29215
#events requiring at least 3 prompt leptons: 61889(-3%)
========= TAU SELECTION ====================
number of taus before selection: 161788
number of taus after selection: 73666
#events requiring at least 3 prompt leptons: 15038(-75%)
#events requiring exactly 3 prompt leptons: 15030(-0%)
==============no bjet ================
#events requiring 0 non-overlapping bjet: 14625(-2%)
percentage of left viable events: 21%


In [5]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\HeavyNeutrino_trilepton\\HeavyNeutrino_trilepton_M-300_V-0p01_tau_NLO_tauhFilter_TuneCP5_13TeV-madgraph-pythia8"
samples = files_from_dir(local_dir),
i_file = 1
tot_events = 0
file = samples[0][0]
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
print("initial number of events: " + str(len(events)))
events["prompt_e"] = events.Electron[events.Electron.genPartFlav == 1]
events["prompt_m"] = events.Muon[events.Muon.genPartFlav == 1]
match_prompt_tau_to_h = events.Tau.genPartFlav == 5
match_prompt_tau_to_e = events.Tau.genPartFlav == 3
match_prompt_tau_to_m = events.Tau.genPartFlav == 4
match_prompt_tau_prompt_e = events.Tau.genPartFlav == 1
match_prompt_tau_prompt_m = events.Tau.genPartFlav == 2
events["prompt_t"] = events.Tau[match_prompt_tau_to_h|match_prompt_tau_to_e|match_prompt_tau_to_m|match_prompt_tau_prompt_e|match_prompt_tau_prompt_m]


events["prompt_l"] = ak.concatenate([events.prompt_e,events.prompt_m, events.prompt_t], axis = -1)
events = events[ak.num(events.prompt_l) ==4]
print(events.prompt_l)
print(events.prompt_l.genPartIdx)
print(events.prompt_l.genPartFlav)

initial number of events: 96498
[[Muon, Tau, Tau, Tau], [Muon, Tau, Tau, ... Tau, Tau, Tau], [Muon, Tau, Tau, Tau]]
[[13, 0, 20, 11], [12, 10, 1, 0], [11, 22, ... 0, 1], [8, 8, 0, 18], [13, 0, 27, 11]]
[[1, 5, 3, 2], [1, 2, 5, 5], [1, 3, 5, 2, ... 1, 5, 5], [1, 2, 5, 3], [1, 5, 4, 2]]


In [4]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\HeavyNeutrino_trilepton\\HeavyNeutrino_trilepton_M-300_V-0p01_tau_NLO_tauhFilter_TuneCP5_13TeV-madgraph-pythia8"
samples = files_from_dir(local_dir),
i_file = 1
tot_events = 0


for file in samples[0]:
    events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
    print(str(i_file)+"/"+str(len(samples[0])))
    i_file = i_file+1
    nb_events = len(events)
    tot_events = tot_events+nb_events

    events["Electron_true"] = events.Electron[events.Electron.genPartIdx >=0]
    events["Electron_true"] = events.Electron_true[abs(events.GenPart[events.Electron_true.genPartIdx].pdgId) == 11]

    events["Electron_from_q"] = events.Electron_true[abs(events.GenPart[events.Electron_true.genPartIdx].distinctParent.pdgId) <= 6]
    events["Electron_from_q"] = events.Electron_from_q[ak.any(abs(events.GenPart[events.Electron_from_q.genPartIdx].distinctParent.distinctChildren.pdgId) == 9900012, axis = -1)]    
    events["Electron_from_W"] = events.Electron_true[abs(events.GenPart[events.Electron_true.genPartIdx].distinctParent.pdgId) == 24]
    events["Electron_from_HNL"] = events.Electron_true[abs(events.GenPart[events.Electron_true.genPartIdx].distinctParent.pdgId) == 9900012]

    events["Muon_true"] = events.Muon[abs(events.GenPart[events.Muon.genPartIdx].pdgId) == 13]
    events["Muon_from_q"] = events.Muon_true[abs(events.GenPart[events.Muon_true.genPartIdx].distinctParent.pdgId) <= 6]
    events["Muon_from_q"] = events.Muon_from_q[ak.any(abs(events.GenPart[events.Muon_from_q.genPartIdx].distinctParent.distinctChildren.pdgId) == 9900012, axis = -1)]
    events["Muon_from_W"] = events.Muon_true[abs(events.GenPart[events.Muon_true.genPartIdx].distinctParent.pdgId) == 24]
    events["Muon_from_HNL"] = events.Muon_true[abs(events.GenPart[events.Muon_true.genPartIdx].distinctParent.pdgId) == 9900012]

    to_h = events.Tau.genPartFlav == 5
    events["SelTau"] = events.Tau[to_h]
    events["SelTau"] = events.SelTau[events.SelTau.genPartIdx>=0]
    events["SelTau"] = events.SelTau[events.SelTau.genPartIdx <ak.num(events.GenVisTau)]
    events["SelTau"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.pdgId) == 15]
    events["Tau_from_q"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.distinctParent.pdgId) <=6]
    events["Tau_from_q"] = events.Tau_from_q[ak.any(abs(events.GenVisTau[events.Tau_from_q.genPartIdx].parent.distinctParent.distinctChildren.pdgId) == 9900012, axis = -1)]
    events["Tau_from_W"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.distinctParent.pdgId) == 24]
    events["Tau_from_HNL"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.distinctParent.pdgId) == 9900012]    
    

    to_m = events.Tau.genPartFlav == 4
    to_e = events.Tau.genPartFlav == 3
    false_m = events.Tau.genPartFlav == 2
    false_e = events.Tau.genPartFlav == 1
    events["SelTau2"] = events.Tau[to_m | to_e]
    #events["SelTau2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.pdgId) == 15]
    events["Tau_from_q2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.distinctParent.pdgId) <=6]
    events["Tau_from_q2"] = events.Tau_from_q2[ak.any(abs(events.GenPart[events.Tau_from_q2.genPartIdx].parent.distinctParent.distinctChildren.pdgId) == 9900012, axis = -1)]
    events["Tau_from_W2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.distinctParent.pdgId) == 24]
    events["Tau_from_HNL2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.distinctParent.pdgId) == 9900012]    

    
    events["Tau_from_unknown"] = events.Tau[~(to_h | to_e | to_m)]
    print(events.Tau_from_unknown)




    events["Lepton_from_q"] = ak.concatenate([events.Electron_from_q, events.Muon_from_q,events.Tau_from_q, events.Tau_from_q2], axis = -1)
    events["Lepton_from_W"] = ak.concatenate([events.Electron_from_W, events.Muon_from_W, events.Tau_from_W, events.Tau_from_W2], axis = -1)
    events["Lepton_from_HNL"] = ak.concatenate([events.Electron_from_HNL, events.Muon_from_HNL, events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)

    print("events with prompt lepton from q : " + str(ak.count_nonzero(ak.num(events.Lepton_from_q)>0)))
    print("events with prompt lepton from W : " + str(ak.count_nonzero(ak.num(events.Lepton_from_W)>0)))
    print("events with prompt lepton from HNL : " + str(ak.count_nonzero(ak.num(events.Lepton_from_HNL)>0)))
    print("----------------")
    l_q = ak.num(events.Lepton_from_q) >= 1
    l_W = ak.num(events.Lepton_from_W) >= 1
    l_HNL = ak.num(events.Lepton_from_HNL) >= 1
    
    leptons_all = l_q & l_W & l_HNL
    print("no selection (but contain the 3 prompt leptons): ")
    no_sel = ak.count_nonzero(leptons_all)
    print(no_sel)
    


####
####
#### selection of electron

    events["Electron_from_q"] = events.Electron_from_q[(events.Electron_from_q.pt > 24.) & (events.Electron_from_q.mvaFall17V2Iso_WP90 > 0.5)]    
    events["Electron_from_W"] = events.Electron_from_W[(events.Electron_from_W.pt > 24.) & (events.Electron_from_W.mvaFall17V2Iso_WP90 > 0.5)]    
    events["Electron_from_HNL"] = events.Electron_from_HNL[(events.Electron_from_HNL.pt > 24.) & (events.Electron_from_HNL.mvaFall17V2Iso_WP90 > 0.5)]    

    events["Lepton_from_q"] = ak.concatenate([events.Electron_from_q, events.Muon_from_q,events.Tau_from_q, events.Tau_from_q2], axis = -1)
    events["Lepton_from_W"] = ak.concatenate([events.Electron_from_W, events.Muon_from_W, events.Tau_from_W, events.Tau_from_W2], axis = -1)
    events["Lepton_from_HNL"] = ak.concatenate([events.Electron_from_HNL, events.Muon_from_HNL, events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)

    l_q = ak.num(events.Lepton_from_q) >= 1
    l_W = ak.num(events.Lepton_from_W) >= 1
    l_HNL = ak.num(events.Lepton_from_HNL) >= 1
    
    leptons_all = l_q & l_W & l_HNL
    print("after selection of electron: ")
    e_sel = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(e_sel/no_sel)))+"%)")

####
####
#### selection of muon

    events["Muon_from_q"] = events.Muon_from_q[(events.Muon_from_q.pt > 24.) & (events.Muon_from_q.mediumPromptId) & (events.Muon_from_q.pfRelIso03_all < 0.2) & (np.abs(events.Muon_from_q.dxy) < 0.005)]    
    events["Muon_from_W"] = events.Muon_from_W[(events.Muon_from_W.pt > 24.) & (events.Muon_from_W.mediumPromptId) & (events.Muon_from_W.pfRelIso03_all < 0.2) & (np.abs(events.Muon_from_W.dxy) < 0.005)]    
    events["Muon_from_HNL"] = events.Muon_from_HNL[(events.Muon_from_HNL.pt > 24.) & (events.Muon_from_HNL.mediumPromptId) & (events.Muon_from_HNL.pfRelIso03_all < 0.2) & (np.abs(events.Muon_from_HNL.dxy) < 0.005)]    

    events["Lepton_from_q"] = ak.concatenate([events.Electron_from_q, events.Muon_from_q,events.Tau_from_q, events.Tau_from_q2], axis = -1)
    events["Lepton_from_W"] = ak.concatenate([events.Electron_from_W, events.Muon_from_W, events.Tau_from_W, events.Tau_from_W2], axis = -1)
    events["Lepton_from_HNL"] = ak.concatenate([events.Electron_from_HNL, events.Muon_from_HNL, events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)

    l_q = ak.num(events.Lepton_from_q) >= 1
    l_W = ak.num(events.Lepton_from_W) >= 1
    l_HNL = ak.num(events.Lepton_from_HNL) >= 1
    
    leptons_all = l_q & l_W & l_HNL
    print("after selection of muon: ")
    mu_sel = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(mu_sel/e_sel)))+"%)")



####
####
#### basic selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["Tau_from_q"] = events.Tau_from_q[(events.Tau_from_q.pt > pt_tau) & (abs(events.Tau_from_q.eta) < 2.3)]
    events["Tau_from_W"] = events.Tau_from_W[(events.Tau_from_W.pt > pt_tau) & (abs(events.Tau_from_W.eta) < 2.3)]
    events["Tau_from_HNL"] = events.Tau_from_HNL[(events.Tau_from_HNL.pt > pt_tau) & (abs(events.Tau_from_HNL.eta) < 2.3)]
    
    events["Tau_from_q2"] = events.Tau_from_q2[(events.Tau_from_q2.pt > pt_tau) & (abs(events.Tau_from_q2.eta) < 2.3)]
    events["Tau_from_W2"] = events.Tau_from_W2[(events.Tau_from_W2.pt > pt_tau) & (abs(events.Tau_from_W2.eta) < 2.3)]
    events["Tau_from_HNL2"] = events.Tau_from_HNL2[(events.Tau_from_HNL2.pt > pt_tau) & (abs(events.Tau_from_HNL2.eta) < 2.3)]
    

    events["Lepton_from_q"] = ak.concatenate([events.Electron_from_q, events.Muon_from_q,events.Tau_from_q, events.Tau_from_q2], axis = -1)
    events["Lepton_from_W"] = ak.concatenate([events.Electron_from_W, events.Muon_from_W, events.Tau_from_W, events.Tau_from_W2], axis = -1)
    events["Lepton_from_HNL"] = ak.concatenate([events.Electron_from_HNL, events.Muon_from_HNL, events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)

    l_q = ak.num(events.Lepton_from_q) == 1
    l_W = ak.num(events.Lepton_from_W) == 1
    l_HNL = ak.num(events.Lepton_from_HNL) == 1
    
    leptons_all = l_q & l_W & l_HNL
    print("after basic selection of tau (pt, eta): ")
    tau_sel1 = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(tau_sel1/mu_sel)))+"%)")

####
####
#### selection of Tau ID vs jet
    id_tauvsjet = 0.5
    pt_tau = 20
    events["Tau_from_q"] = events.Tau_from_q[(events.Tau_from_q.pt > pt_tau) & (abs(events.Tau_from_q.eta) < 2.3)& (events.Tau_from_q.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_W"] = events.Tau_from_W[(events.Tau_from_W.pt > pt_tau) & (abs(events.Tau_from_W.eta) < 2.3)& (events.Tau_from_W.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_HNL"] = events.Tau_from_HNL[(events.Tau_from_HNL.pt > pt_tau) & (abs(events.Tau_from_HNL.eta) < 2.3) & (events.Tau_from_HNL.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    
    events["Tau_from_q2"] = events.Tau_from_q2[(events.Tau_from_q2.pt > pt_tau) & (abs(events.Tau_from_q2.eta) < 2.3) & (events.Tau_from_q2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_W2"] = events.Tau_from_W2[(events.Tau_from_W2.pt > pt_tau) & (abs(events.Tau_from_W2.eta) < 2.3)& (events.Tau_from_W2.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau_from_W2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_HNL2"] = events.Tau_from_HNL2[(events.Tau_from_HNL2.pt > pt_tau) & (abs(events.Tau_from_HNL2.eta) < 2.3)& (events.Tau_from_HNL2.idDeepTau2017v2p1VSmu > 0.5)  & (events.Tau_from_HNL2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    

    events["Lepton_from_q"] = ak.concatenate([events.Electron_from_q, events.Muon_from_q,events.Tau_from_q, events.Tau_from_q2], axis = -1)
    events["Lepton_from_W"] = ak.concatenate([events.Electron_from_W, events.Muon_from_W, events.Tau_from_W, events.Tau_from_W2], axis = -1)
    events["Lepton_from_HNL"] = ak.concatenate([events.Electron_from_HNL, events.Muon_from_HNL, events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)

    l_q = ak.num(events.Lepton_from_q) >= 1
    l_W = ak.num(events.Lepton_from_W) >= 1
    l_HNL = ak.num(events.Lepton_from_HNL) >= 1
    
    leptons_all = l_q & l_W & l_HNL
    print("after ID selection of tau vs jet: ")
    tau_sel2 = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(tau_sel2/tau_sel1)))+"%)")
 


####
####
#### selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["Tau_from_q"] = events.Tau_from_q[(events.Tau_from_q.pt > pt_tau) & (abs(events.Tau_from_q.eta) < 2.3) & (events.Tau_from_q.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_q.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_W"] = events.Tau_from_W[(events.Tau_from_W.pt > pt_tau) & (abs(events.Tau_from_W.eta) < 2.3) & (events.Tau_from_W.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_W.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_HNL"] = events.Tau_from_HNL[(events.Tau_from_HNL.pt > pt_tau) & (abs(events.Tau_from_HNL.eta) < 2.3) & (events.Tau_from_HNL.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_HNL.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    
    events["Tau_from_q2"] = events.Tau_from_q2[(events.Tau_from_q2.pt > pt_tau) & (abs(events.Tau_from_q2.eta) < 2.3)& (events.Tau_from_q2.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_q2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_W2"] = events.Tau_from_W2[(events.Tau_from_W2.pt > pt_tau) & (abs(events.Tau_from_W2.eta) < 2.3) & (events.Tau_from_W2.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_W2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_HNL2"] = events.Tau_from_HNL2[(events.Tau_from_HNL2.pt > pt_tau) & (abs(events.Tau_from_HNL2.eta) < 2.3)& (events.Tau_from_HNL2.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_HNL2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    

    events["Lepton_from_q"] = ak.concatenate([events.Electron_from_q, events.Muon_from_q,events.Tau_from_q, events.Tau_from_q2], axis = -1)
    events["Lepton_from_W"] = ak.concatenate([events.Electron_from_W, events.Muon_from_W, events.Tau_from_W, events.Tau_from_W2], axis = -1)
    events["Lepton_from_HNL"] = ak.concatenate([events.Electron_from_HNL, events.Muon_from_HNL, events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)

    l_q = ak.num(events.Lepton_from_q) >= 1
    l_W = ak.num(events.Lepton_from_W) >= 1
    l_HNL = ak.num(events.Lepton_from_HNL) >= 1
    
    leptons_all = l_q & l_W & l_HNL
    print("after ID selection of tau vs e: ")
    tau_sel3 = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(tau_sel3/tau_sel2)))+"%)")
 


####
####
#### selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["Tau_from_q"] = events.Tau_from_q[(events.Tau_from_q.pt > pt_tau) & (abs(events.Tau_from_q.eta) < 2.3)& (events.Tau_from_q.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau_from_q.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_q.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_W"] = events.Tau_from_W[(events.Tau_from_W.pt > pt_tau) & (abs(events.Tau_from_W.eta) < 2.3)& (events.Tau_from_W.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau_from_W.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_W.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_HNL"] = events.Tau_from_HNL[(events.Tau_from_HNL.pt > pt_tau) & (abs(events.Tau_from_HNL.eta) < 2.3)& (events.Tau_from_HNL.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau_from_HNL.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_HNL.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    
    events["Tau_from_q2"] = events.Tau_from_q2[(events.Tau_from_q2.pt > pt_tau) & (abs(events.Tau_from_q2.eta) < 2.3)& (events.Tau_from_q2.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau_from_q2.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_q2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_W2"] = events.Tau_from_W2[(events.Tau_from_W2.pt > pt_tau) & (abs(events.Tau_from_W2.eta) < 2.3)& (events.Tau_from_W2.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau_from_W2.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_W2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events["Tau_from_HNL2"] = events.Tau_from_HNL2[(events.Tau_from_HNL2.pt > pt_tau) & (abs(events.Tau_from_HNL2.eta) < 2.3)& (events.Tau_from_HNL2.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau_from_HNL2.idDeepTau2017v2p1VSe > 0.5) & (events.Tau_from_HNL2.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    

    events["Lepton_from_q"] = ak.concatenate([events.Electron_from_q, events.Muon_from_q,events.Tau_from_q, events.Tau_from_q2], axis = -1)
    events["Lepton_from_W"] = ak.concatenate([events.Electron_from_W, events.Muon_from_W, events.Tau_from_W, events.Tau_from_W2], axis = -1)
    events["Lepton_from_HNL"] = ak.concatenate([events.Electron_from_HNL, events.Muon_from_HNL, events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)

    l_q = ak.num(events.Lepton_from_q) >= 1
    l_W = ak.num(events.Lepton_from_W) >= 1
    l_HNL = ak.num(events.Lepton_from_HNL) >= 1
    
    leptons_all = l_q & l_W & l_HNL
    print("after ID selection of tau: ")
    tau_sel4 = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(tau_sel4/tau_sel3)))+"%)")
 

    l_q = ak.num(events.Lepton_from_q) == 1
    l_W = ak.num(events.Lepton_from_W) == 1
    l_HNL = ak.num(events.Lepton_from_HNL) == 1
    
    leptons_all = l_q & l_W & l_HNL
    print("exactly 3 leptons: ")
    three_sel = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(three_sel/tau_sel4)))+"%)")





####
####
#### no bjet

    events["Lepton_from_all"] = ak.concatenate([events.Lepton_from_q, events.Lepton_from_W, events.Lepton_from_HNL], axis = -1)
    jets = events.Jet[events.Jet.pt > 25.]
    bjets = jets[jets.btagDeepFlavB > 0.2770]
    events["bjet"] = events.Jet[events.Jet.pt > 25.]
    events["bjet"] = events.bjet[events.bjet.btagDeepFlavB > 0.2770]
    bjets_temp, l = ak.unzip(ak.cartesian([events.bjet, events.Lepton_from_all], nested=True))
    overlap = ak.any(bjets_temp.delta_r(l) < 0.5, axis=-1, mask_identity=False)
    events["bjet"] = events.bjet[~overlap]
    no_jet = ak.num(events.bjet) == 0
    leptons_all = l_q & l_W & l_HNL & no_jet
    print("exactly 3 leptons and no non-overlapping bjet: ")

    jet_sel = ak.count_nonzero(leptons_all)
    print(str(ak.count_nonzero(leptons_all)) + "(-"+str(int(100 -100*(jet_sel/three_sel)))+"%)")


    print("to recall, all events: ")
    print(len(events))

    all_events = len(events)
    print("percentage of signal events after all selection over total events: "  + "("+str(int(100*(jet_sel/all_events)))+"%)")
    print("percentage of signal events after all selection over 'usable' events: "  + "("+str(int(100*(jet_sel/no_sel)))+"%)")





1/1
[[Tau], [Tau], [Tau], [Tau, Tau], [Tau], ... [], [Tau], [Tau], [Tau, Tau], [], []]
events with prompt lepton from q : 66642
events with prompt lepton from W : 73380
events with prompt lepton from HNL : 59675
----------------
no selection (but contain the 3 prompt leptons): 
34492
after selection of electron: 
31874(-7%)
after selection of muon: 
29719(-6%)
after basic selection of tau (pt, eta): 
26318(-11%)
after ID selection of tau vs jet: 
21444(-18%)
after ID selection of tau vs e: 
14854(-30%)
after ID selection of tau: 
11455(-22%)
exactly 3 leptons: 
11438(-0%)
exactly 3 leptons and no non-overlapping bjet: 
11145(-2%)
to recall, all events: 
96498
percentage of signal events after all selection over total events: (11%)
percentage of signal events after all selection over 'usable' events: (32%)


In [4]:
###FOR TTBAR BACKGROUND

import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\backgrounds\\tt2l2nu"
samples = files_from_dir(local_dir),
i_file = 1
tot_events = 0
print("For ttbar:")

for file in samples[0]:
    events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
    print(str(i_file)+"/"+str(len(samples[0])))
    i_file = i_file+1
    nb_events = len(events)
    tot_events = tot_events+nb_events

    events["SelElectron"] = events.Electron
    events["SelMuon"] = events.Muon 
    events["SelTau"] = events.Tau

    print("no selection : ")
    no_sel = len(events)
    print(no_sel)
    

    print("Require at least 3 leptons")
    events_temp = events[ak.num(events.SelTau) + ak.num(events.SelMuon) + ak.num(events.SelElectron) >= 3]
    print(len(events_temp))

####
####
#### selection of electron

    events["SelElectron"] = events.Electron[(events.Electron.pt > 24.) & (events.Electron.mvaFall17V2Iso_WP90 > 0.5)]    
    events = events[ak.num(events.SelElectron) <= 3]
    print("after selection of electron (nb.e <=3): ")
    e_sel = len(events)
    print(str(e_sel) + "(-"+str(int(100 -100*(e_sel/no_sel)))+"%)")

####
####
#### selection of muon


    events["SelMuon"] = events.SelMuon[(events.SelMuon.pt > 24.) & (events.SelMuon.mediumPromptId) & (events.SelMuon.pfRelIso03_all < 0.2) & (np.abs(events.SelMuon.dxy) < 0.005)]       
    events = events[ak.num(events.SelMuon) <= 3]
    print("after selection of Muon(nb.mu <=3): ")
    mu_sel = len(events)
    print(str(mu_sel) + "(-"+str(int(100 -100*(mu_sel/e_sel)))+"%)")



####
####
#### basic selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3)]

    events = events[ak.num(events.SelTau) <= 3]
    print("after basic selection of tau (pt, eta) (nb.tau <=3): ")
    tau_sel1 = len(events)
    print(str(tau_sel1) + "(-"+str(int(100 -100*(tau_sel1/mu_sel)))+"%)")

####
####
#### selection of Tau ID vs jet
    id_tauvsjet = 0.5
    pt_tau = 20 
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3) & (events.SelTau.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events = events[ak.num(events.SelTau) <= 3]
    print("after ID selection of tau (vs jet) (nb.tau <=3): ")
    tau_sel2 = len(events)
    print(str(tau_sel2) + "(-"+str(int(100 -100*(tau_sel2/tau_sel1)))+"%)")


####
####
#### selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3) & (events.SelTau.idDeepTau2017v2p1VSe > 0.5) & (events.SelTau.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events = events[ak.num(events.SelTau) <= 3]
    print("after ID selection of tau (vs e) (nb.tau <=3): ")
    tau_sel3 = len(events)
    print(str(tau_sel3) + "(-"+str(int(100 -100*(tau_sel3/tau_sel2)))+"%)")



####
####
#### selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3)& (events.SelTau.idDeepTau2017v2p1VSe > 0.5) & (events.SelTau.idDeepTau2017v2p1VSe > 0.5) & (events.SelTau.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events = events[ak.num(events.SelTau) <= 3]
    print("after ID selection of tau (vs mu) (nb.tau <=3): ")
    tau_sel4 = len(events)
    print(str(tau_sel4) + "(-"+str(int(100 -100*(tau_sel4/tau_sel3)))+"%)")



    events = events[ak.num(events.SelTau) + ak.num(events.SelMuon) + ak.num(events.SelElectron) == 3]
    end_of_sel = len(events)
    print("requiring exactly 3 leptons")
    print(str(end_of_sel) + "(-"+str(int(100 -100*(end_of_sel/tau_sel4)))+"%)")

    ####BJET veto
    events["SelLeptons"] = ak.concatenate([events.SelElectron, events.SelMuon, events.SelTau], axis = -1)
    jets = events.Jet[events.Jet.pt > 25.]
    bjets = jets[jets.btagDeepFlavB > 0.2770]
    events["bjet"] = events.Jet[events.Jet.pt > 25.]
    events["bjet"] = events.bjet[events.bjet.btagDeepFlavB > 0.2770]
    bjets_temp, l = ak.unzip(ak.cartesian([events.bjet, events.SelLeptons], nested=True))
    overlap = ak.any(bjets_temp.delta_r(l) < 0.5, axis=-1, mask_identity=False)
    events["bjet"] = events.bjet[~overlap]
    events = events[ak.num(events.bjet) == 0]
    
    jet_sel = len(events)
    print(str(jet_sel) + "(-"+str(int(100 -100*(jet_sel/end_of_sel)))+"%)")


    print("to recall, all events: ")
    print(no_sel)

    print("percentage of remaining bkg events after all selection over total events: "  + "("+str(int(100*(jet_sel/no_sel)))+"%)")

    


For ttbar:
1/1
no selection : 
2025984
Require at least 3 leptons
2025984
after selection of electron (nb.e <=3): 
2025984(-0%)
after selection of Muon(nb.mu <=3): 
2025961(-0%)
after basic selection of tau (pt, eta) (nb.tau <=3): 
1797502(-11%)
after ID selection of tau (vs jet) (nb.tau <=3): 
1797502(-0%)
after ID selection of tau (vs e) (nb.tau <=3): 
1797502(-0%)
after ID selection of tau (vs mu) (nb.tau <=3): 
1797502(-0%)
requiring exactly 3 leptons
228866(-87%)
44566(-80%)
to recall, all events: 
2025984
percentage of remaining bkg events after all selection over total events: (2%)


In [6]:
###FOR WZ BACKGROUND

import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\backgrounds\\wz"
samples = files_from_dir(local_dir),
i_file = 1
tot_events = 0
print("For wz:")

for file in samples[0]:
    events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
    print(str(i_file)+"/"+str(len(samples[0])))
    i_file = i_file+1
    nb_events = len(events)
    tot_events = tot_events+nb_events

    events["SelElectron"] = events.Electron
    events["SelMuon"] = events.Muon 
    events["SelTau"] = events.Tau

    print("no selection : ")
    no_sel = len(events)
    print(no_sel)
    

    print("Require at least 3 leptons")
    events_temp = events[ak.num(events.SelTau) + ak.num(events.SelMuon) + ak.num(events.SelElectron) >= 3]
    print(len(events_temp))

####
####
#### selection of electron

    events["SelElectron"] = events.Electron[(events.Electron.pt > 24.) & (events.Electron.mvaFall17V2Iso_WP90 > 0.5)]    
    events = events[ak.num(events.SelElectron) <= 3]
    print("after selection of electron (nb.e <=3): ")
    e_sel = len(events)
    print(str(e_sel) + "(-"+str(int(100 -100*(e_sel/no_sel)))+"%)")

####
####
#### selection of muon


    events["SelMuon"] = events.SelMuon[(events.SelMuon.pt > 24.) & (events.SelMuon.mediumPromptId) & (events.SelMuon.pfRelIso03_all < 0.2) & (np.abs(events.SelMuon.dxy) < 0.005)]       
    events = events[ak.num(events.SelMuon) <= 3]
    print("after selection of Muon(nb.mu <=3): ")
    mu_sel = len(events)
    print(str(mu_sel) + "(-"+str(int(100 -100*(mu_sel/e_sel)))+"%)")



####
####
#### basic selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3)]

    events = events[ak.num(events.SelTau) <= 3]
    print("after basic selection of tau (pt, eta) (nb.tau <=3): ")
    tau_sel1 = len(events)
    print(str(tau_sel1) + "(-"+str(int(100 -100*(tau_sel1/mu_sel)))+"%)")

####
####
#### selection of Tau ID vs jet
    id_tauvsjet = 0.5
    pt_tau = 20 
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3) & (events.SelTau.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events = events[ak.num(events.SelTau) <= 3]
    print("after ID selection of tau (vs jet) (nb.tau <=3): ")
    tau_sel2 = len(events)
    print(str(tau_sel2) + "(-"+str(int(100 -100*(tau_sel2/tau_sel1)))+"%)")


####
####
#### selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3) & (events.SelTau.idDeepTau2017v2p1VSe > 0.5) & (events.SelTau.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events = events[ak.num(events.SelTau) <= 3]
    print("after ID selection of tau (vs e) (nb.tau <=3): ")
    tau_sel3 = len(events)
    print(str(tau_sel3) + "(-"+str(int(100 -100*(tau_sel3/tau_sel2)))+"%)")



####
####
#### selection of Tau
    id_tauvsjet = 0.5
    pt_tau = 20
    events["SelTau"] = events.SelTau[(events.SelTau.pt > pt_tau) & (abs(events.SelTau.eta) < 2.3)& (events.SelTau.idDeepTau2017v2p1VSe > 0.5) & (events.SelTau.idDeepTau2017v2p1VSe > 0.5) & (events.SelTau.idDeepTau2017v2p1VSjet >=id_tauvsjet)]
    events = events[ak.num(events.SelTau) <= 3]
    print("after ID selection of tau (vs mu) (nb.tau <=3): ")
    tau_sel4 = len(events)
    print(str(tau_sel4) + "(-"+str(int(100 -100*(tau_sel4/tau_sel3)))+"%)")



    events = events[ak.num(events.SelTau) + ak.num(events.SelMuon) + ak.num(events.SelElectron) == 3]
    end_of_sel = len(events)
    print("requiring exactly 3 leptons")
    print(str(end_of_sel) + "(-"+str(int(100 -100*(end_of_sel/tau_sel4)))+"%)")

    ####BJET veto
    events["SelLeptons"] = ak.concatenate([events.SelElectron, events.SelMuon, events.SelTau], axis = -1)
    jets = events.Jet[events.Jet.pt > 25.]
    bjets = jets[jets.btagDeepFlavB > 0.2770]
    events["bjet"] = events.Jet[events.Jet.pt > 25.]
    events["bjet"] = events.bjet[events.bjet.btagDeepFlavB > 0.2770]
    bjets_temp, l = ak.unzip(ak.cartesian([events.bjet, events.SelLeptons], nested=True))
    overlap = ak.any(bjets_temp.delta_r(l) < 0.5, axis=-1, mask_identity=False)
    events["bjet"] = events.bjet[~overlap]
    events = events[ak.num(events.bjet) == 0]
    
    jet_sel = len(events)
    print("no overlapping bjet selection ")
    print(str(jet_sel) + "(-"+str(int(100 -100*(jet_sel/end_of_sel)))+"%)")


    print("to recall, all events: ")
    print(no_sel)

    print("percentage of remaining bkg events after all selection over total events: "  + "("+str(int(100*(jet_sel/no_sel)))+"%)")

    


For wz:


C:\Users\lucas\miniconda3\envs\env_conda\envs\PDM\lib\site-packages\coffea\nanoevents\schemas\nanoaod.py:195: RuntimeWarning: Missing cross-reference index for FatJet_genJetAK8Idx => GenJetAK8
  RuntimeWarning,
C:\Users\lucas\miniconda3\envs\env_conda\envs\PDM\lib\site-packages\coffea\nanoevents\schemas\nanoaod.py:195: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx1 => SubJet
  RuntimeWarning,
C:\Users\lucas\miniconda3\envs\env_conda\envs\PDM\lib\site-packages\coffea\nanoevents\schemas\nanoaod.py:195: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx2 => SubJet
  RuntimeWarning,


1/1
no selection : 
454011
Require at least 3 leptons
454011
after selection of electron (nb.e <=3): 
454011(-0%)
after selection of Muon(nb.mu <=3): 
454001(-0%)
after basic selection of tau (pt, eta) (nb.tau <=3): 
418092(-7%)
after ID selection of tau (vs jet) (nb.tau <=3): 
418092(-0%)
after ID selection of tau (vs e) (nb.tau <=3): 
418092(-0%)
after ID selection of tau (vs mu) (nb.tau <=3): 
418092(-0%)
requiring exactly 3 leptons
23028(-94%)
no overlapping bjet selection 
21480(-6%)
to recall, all events: 
454011
percentage of remaining bkg events after all selection over total events: (4%)


In [33]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\HeavyNeutrino_trilepton\\HeavyNeutrino_trilepton_M-300_V-0p01_tau_NLO_tauhFilter_TuneCP5_13TeV-madgraph-pythia8"
samples = files_from_dir(local_dir),


for file in samples[0]:
    events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()

    # events["Tau_genVis_parent"] = events.GenVisTau.parent.distinctParent.pdgId
    # print(events.Tau_genVis_parent[events.Tau_genVis_parent == 9900012])  
    print("Overall total of events = " + str(len(events)))
    print("------")
    print("For HNL lepton")
    HNL = events.GenPart[(np.abs(events.GenPart.pdgId) == 9900012)]
    tau_from_HNL = HNL[HNL.hasFlags(['isLastCopy'])]
    tau_from_HNL = tau_from_HNL.distinctChildren[abs(tau_from_HNL.distinctChildren.pdgId) == 15]
    tau_from_HNL = tau_from_HNL[tau_from_HNL.hasFlags(['isLastCopy']) ]
    tau_to_h = tau_from_HNL[ak.any(abs(tau_from_HNL.distinctChildren.pdgId) > 30, axis = -1)]
    tau_to_e = tau_from_HNL[ak.any(abs(tau_from_HNL.distinctChildren.pdgId) == 11, axis = -1)]
    tau_to_m = tau_from_HNL[ak.any(abs(tau_from_HNL.distinctChildren.pdgId) == 13, axis = -1)]
    print(len(ak.flatten(ak.flatten(tau_to_e))))
    print(len(ak.flatten(ak.flatten(tau_to_m))))
    print(len(ak.flatten(ak.flatten(tau_to_h))))

    


    #acceptance
    tau_kept = tau_from_HNL[tau_from_HNL.pt > 20]
    tau_kept = tau_kept[abs(tau_kept.eta) < 2.3]
    #efficiency
    tau_rec = events.Tau[(events.Tau.pt > 20.) & (abs(events.Tau.eta) < 2.3)& (events.Tau.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau.idDeepTau2017v2p1VSe > 0.5) & (events.Tau.idDeepTau2017v2p1VSjet >=8)]
    
    print("taus: " + str(len(ak.flatten(tau_from_HNL))) + " -> Acceptance -> " + str(len(ak.flatten(tau_kept))) + " ("+str(int(100*len(ak.flatten(tau_kept))/len(ak.flatten(tau_from_HNL))))+"%)")
    print("------")
    print("For W lepton")

    W = events.GenPart[(np.abs(events.GenPart.pdgId) == 24)]
    W = W[W.hasFlags(['isLastCopy'])]
    W = W[abs(W.distinctParent.pdgId) == 9900012]
    tau_from_W = ak.flatten(W.distinctChildren[abs(W.distinctChildren.pdgId) == 15])
    electron_from_W = ak.flatten(W.distinctChildren[abs(W.distinctChildren.pdgId) == 11])
    muon_from_W = ak.flatten(W.distinctChildren[abs(W.distinctChildren.pdgId) == 13])
    muon_from_W = muon_from_W[muon_from_W.hasFlags(['isLastCopy'])]
    electron_from_W = electron_from_W[electron_from_W.hasFlags(['isLastCopy'])]
    tau_from_W = tau_from_W[tau_from_W.hasFlags(['isLastCopy'])]
    tau_kept = tau_from_W[tau_from_W.pt > 20]
    tau_kept = tau_kept[abs(tau_kept.eta) < 2.3]
    electron_kept = electron_from_W[electron_from_W.pt > 20]
    muon_kept = muon_from_W[muon_from_W.pt > 20]
    print("taus: " + str(len(ak.flatten(tau_from_W))) + " -> Acceptance -> " + str(len(ak.flatten(tau_kept))) + " ("+str(int(100*len(ak.flatten(tau_kept))/len(ak.flatten(tau_from_W))))+"%)")
    print("electrons: " + str(len(ak.flatten(electron_from_W))) + " -> Acceptance -> " + str(len(ak.flatten(electron_kept))) + " ("+str(int(100*len(ak.flatten(electron_kept))/len(ak.flatten(electron_from_W))))+"%)")
    print("muons: " + str(len(ak.flatten(muon_from_W))) + " -> Acceptance -> " + str(len(ak.flatten(muon_kept))) + " ("+str(int(100*len(ak.flatten(muon_kept))/len(ak.flatten(muon_from_W))))+"%)")
    print("all: " + str(len(ak.flatten(muon_from_W)) + len(ak.flatten(electron_from_W)) + len(ak.flatten(tau_from_W))) + " -> Acceptance -> " + str(len(ak.flatten(muon_kept)) + len(ak.flatten(tau_kept)) + len(ak.flatten(electron_kept)))  + " ("+str(int(100*(len(ak.flatten(muon_kept)) + len(ak.flatten(tau_kept)) + len(ak.flatten(electron_kept)))/(len(ak.flatten(muon_from_W)) + len(ak.flatten(electron_from_W)) + len(ak.flatten(tau_from_W)))))+"%)" )

 

    print("------")
    print("For q lepton")

    q = events.GenPart[(abs(events.GenPart.pdgId) <= 9) ]
    q = q[q.hasFlags(['isLastCopy'])]
    q = q[ak.any(abs(q.distinctChildren.pdgId) == 9900012, axis = -1)]
    tau_from_q = q.distinctChildren[abs(q.distinctChildren.pdgId )== 15]
    tau_from_q = ak.flatten(tau_from_q[tau_from_q.hasFlags(['isLastCopy'])])
    electron_from_q = q.distinctChildren[abs(q.distinctChildren.pdgId )== 11]
    electron_from_q = ak.flatten(electron_from_q[electron_from_q.hasFlags(['isLastCopy'])])
    muon_from_q = q.distinctChildren[abs(q.distinctChildren.pdgId )== 13]
    muon_from_q = ak.flatten(muon_from_q[muon_from_q.hasFlags(['isLastCopy'])])
    tau_kept = tau_from_q[tau_from_q.pt > 20]
    tau_kept = tau_kept[abs(tau_kept.eta) < 2.3]
    electron_kept = electron_from_q[electron_from_q.pt > 20]
    muon_kept = muon_from_q[muon_from_q.pt > 20]
    print("taus: " + str(len(ak.flatten(tau_from_q))) + " -> Acceptance -> " + str(len(ak.flatten(tau_kept))) + " ("+str(int(100*len(ak.flatten(tau_kept))/len(ak.flatten(tau_from_q))))+"%)")
    #print("electrons: " + str(len(ak.flatten(electron_from_q))) + " -> Acceptance -> " + str(len(ak.flatten(electron_kept))) + " ("+str(int(100*len(ak.flatten(electron_kept))/len(ak.flatten(electron_from_q))))+"%)")
    #print("muons: " + str(len(ak.flatten(muon_from_q))) + " -> Acceptance -> " + str(len(ak.flatten(muon_kept))) + " ("+str(int(100*len(ak.flatten(muon_kept))/len(ak.flatten(muon_from_W))))+"%)")
    #print("all: " + str(len(ak.flatten(muon_from_q)) + len(ak.flatten(electron_from_q)) + len(ak.flatten(tau_from_q))) + " -> Acceptance -> " + str(len(ak.flatten(muon_kept)) + len(ak.flatten(tau_kept)) + len(ak.flatten(electron_kept)))  + " ("+str(int(100*(len(ak.flatten(muon_kept)) + len(ak.flatten(tau_kept)) + len(ak.flatten(electron_kept)))/(len(ak.flatten(muon_from_q)) + len(ak.flatten(electron_from_q)) + len(ak.flatten(tau_from_q)))))+"%)" )
    print("(The other 3K events are taus coming from gluons)")



Overall total of events = 96498
------
For HNL lepton
[[[[16, -211, -211, 211]]], [[[16, 111, ... 211, -211]]], [[[-16, 211, 211, -211]]]]
12957
12586
61015
taus: 96498 -> Acceptance -> 96498 (100%)
------
For W lepton
taus: 20081 -> Acceptance -> 17159 (85%)
electrons: 33307 -> Acceptance -> 30756 (92%)
muons: 33217 -> Acceptance -> 30829 (92%)
all: 86605 -> Acceptance -> 78744 (90%)
------
For q lepton
taus: 93616 -> Acceptance -> 81633 (87%)
(The other 3K events are taus coming from gluons)


In [18]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\HeavyNeutrino_trilepton\\HeavyNeutrino_trilepton_M-300_V-0p01_tau_NLO_tauhFilter_TuneCP5_13TeV-madgraph-pythia8"
samples = files_from_dir(local_dir),


for file in samples[0]:
    events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
    Taus_to_h = events.Tau[events.Tau.genPartFlav == 5]
    print(len(ak.flatten(Taus_to_h.genPartIdx)))
    Taus_to_h = Taus_to_h[Taus_to_h.genPartIdx<5]
    print(len(ak.flatten(Taus_to_h.genPartIdx)))

    to_m = events.Tau.genPartFlav == 4
    to_e = events.Tau.genPartFlav == 3
    events["Taus_to_l"] = events.Tau[to_m |to_e]
    temp = events.GenPart[events.Taus_to_l.genPartIdx].parent.pdgId
    print(np.unique(ak.flatten(temp)))


101003
100974
[-15, 15]


In [46]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\HeavyNeutrino_trilepton\\HeavyNeutrino_trilepton_M-300_V-0p01_tau_NLO_tauhFilter_TuneCP5_13TeV-madgraph-pythia8"
samples = files_from_dir(local_dir),
i_file = 1
tot_events = 0


for file in samples[0]:
    events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
    events_all = events
    print(str(i_file)+"/"+str(len(samples[0])))
    i_file = i_file+1
    nb_events = len(events)
    tot_events = tot_events+nb_events


    to_h = events.Tau.genPartFlav == 5
    events["SelTau"] = events.Tau[to_h]
    events["SelTau"] = events.SelTau[events.SelTau.genPartIdx>=0]
    events["SelTau"] = events.SelTau[events.SelTau.genPartIdx <ak.num(events.GenVisTau)]
    events["SelTau"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.pdgId) == 15]
    events["Tau_from_q"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.distinctParent.pdgId) <=6]
    events["Tau_from_q"] = events.Tau_from_q[ak.any(abs(events.GenVisTau[events.Tau_from_q.genPartIdx].parent.distinctParent.distinctChildren.pdgId) == 9900012, axis = -1)]
    events["Tau_from_W"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.distinctParent.pdgId) == 24]
    events["Tau_from_HNL"] = events.SelTau[abs(events.GenVisTau[events.SelTau.genPartIdx].parent.distinctParent.pdgId) == 9900012]    
    

    to_m = events.Tau.genPartFlav == 4
    to_e = events.Tau.genPartFlav == 3
    events["SelTau2"] = events.Tau[to_m | to_e]
    events["SelTau2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.pdgId) == 15]
    events["Tau_from_q2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.distinctParent.pdgId) <=6]
    events["Tau_from_q2"] = events.Tau_from_q2[ak.any(abs(events.GenPart[events.Tau_from_q2.genPartIdx].parent.distinctParent.distinctChildren.pdgId) == 9900012, axis = -1)]
    events["Tau_from_W2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.distinctParent.pdgId) == 24]
    events["Tau_from_HNL2"] = events.SelTau2[abs(events.GenPart[events.SelTau2.genPartIdx].parent.distinctParent.pdgId) == 9900012]    

    events["Tau_from_HNL_all"] = ak.concatenate([events.Tau_from_HNL, events.Tau_from_HNL2], axis = -1)
    events = events[ak.num(events.Tau_from_HNL_all) == 0]

    to_h = events.Tau.genPartFlav == 5
    to_m = events.Tau.genPartFlav == 4
    to_e = events.Tau.genPartFlav == 3
    events["Tau_from_unknown"] = events.Tau[~(to_h | to_e | to_m)]

    print("events with no taus coming from HNL: ("+str(len(events))+"/"+str(len(events_all))+")")
    print(events.Tau_from_unknown)
    print("correspondging GenPart ID :")
    print(events.GenPart[events.Tau_from_unknown.genPartIdx].pdgId)
    print("Mother ID:")
    print(events.GenPart[events.Tau_from_unknown.genPartIdx].distinctParent.pdgId)
    print("summary of ID:")
    print(np.unique(ak.to_numpy(ak.flatten(events.GenPart[events.Tau_from_unknown.genPartIdx].distinctParent.pdgId))))
    print("GD Mother ID: ")
    print(events.GenPart[events.Tau_from_unknown.genPartIdx].distinctParent.distinctParent.pdgId)
    print("summary of ID:")
    print(np.unique(ak.to_numpy(ak.flatten(events.GenPart[events.Tau_from_unknown.genPartIdx].distinctParent.distinctParent.pdgId))))
    
    
    temp = events.GenPart[events.Tau_from_unknown.genPartIdx].distinctParent.distinctParent.pdgId
    temp2 =  events.GenPart[events.Tau_from_unknown.genPartIdx].distinctParent.pdgId
    temp = temp[temp == 9900012]
    temp2 = temp2[temp == 9900012]
    print("number of events with tau with HNL grandmother: "+str(ak.count_nonzero(ak.any(temp == 9900012, axis = -1))))
    print("mothers ID of those taus : " + str(np.unique(ak.to_numpy(ak.flatten(temp2)))))



1/1
events with no taus coming from HNL: (36828/96498)
[[Tau, Tau], [Tau], [], [], [Tau], [], ... [Tau], [Tau], [Tau], [], [Tau, Tau]]
correspondging GenPart ID :
[[11, -11], [-11], [], [], [11], [], [-11, ... [], [-13], [13], [-13], [], [-13, 13]]
Mother ID:
[[23, 23], [24], [], [], [-24], [], [24], ... 1], [], [24], [23], [24], [], [23, 23]]
summary of ID:
[-4324 -4122 -521 -511 -435 -433 -431 -425 -423 -421 -413 -411 -323 -311
 -213 -24 -15 -4 1 2 3 4 13 15 21 22 23 24 111 113 221 223 311 323 411 413
 421 423 431 433 511 521 2101 2103 2203 3103 3203 4122 4222 4224 4324 --]
GD Mother ID: 
[[9900012, 9900012], [9900012], [], [], ... [9900012], [], [9900012, 9900012]]
summary of ID:
[-4224 -4112 -531 -523 -521 -513 -511 -433 -431 -423 -421 -415 -413 -411
 -323 -24 -15 -4 -3 -2 -1 1 2 3 4 11 15 21 23 24 221 223 323 411 413 421
 423 425 431 433 443 511 513 521 523 531 2203 4103 4114 4122 4132 4222
 5122 9900012 --]
number of events with tau with HNL grandmother: 24962
mothers ID of those